In [70]:
import pandas as pd
import numpy as np
import datetime
import os

from utils import one_hot_encoder, loadpkl

num_rows=10000

%matplotlib inline

In [31]:
# load datasets

historical_transactions = pd.read_csv('../input/historical_transactions.csv', nrows=num_rows)
merchants = pd.read_csv('../input/merchants.csv', index_col=0)
new_merchant_transactions = pd.read_csv('../input/new_merchant_transactions.csv', index_col=0)
sample_submission = pd.read_csv('../input/sample_submission.csv', index_col=0)
test = pd.read_csv('../input/test.csv', index_col=['card_id'])
train = pd.read_csv('../input/train.csv', index_col=['card_id'])

In [21]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [20]:
merchants.head()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
merchant_id,,,,,,,,,,,,,,,,,,,,,
M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,9.666667,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,1.750000,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,260.000000,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,1.666667,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,0.500000,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [19]:
new_merchant_transactions.head()

,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
authorized_flag,,,,,,,,,,,,,
Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [18]:
sample_submission.head()

,target
card_id,
C_ID_0ab67a22ab,0
C_ID_130fd0cbdd,0
C_ID_b709037bc5,0
C_ID_d27d835a9f,0
C_ID_2b5e3df5c2,0


In [30]:
test.head()

,first_active_month,feature_1,feature_2,feature_3
card_id,,,,
C_ID_0ab67a22ab,2017-04,3,3,1
C_ID_130fd0cbdd,2017-01,2,3,0
C_ID_b709037bc5,2017-08,5,1,1
C_ID_d27d835a9f,2017-12,2,1,0
C_ID_2b5e3df5c2,2015-12,5,1,1


In [34]:
train.head()

,first_active_month,feature_1,feature_2,feature_3,target
card_id,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283
C_ID_3d0044924f,2017-01,4,1,0,0.392913
C_ID_d639edf6cd,2016-08,2,2,0,0.688056
C_ID_186d6a6901,2017-09,4,3,0,0.142495
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749


In [67]:
# set target as nan
test['target'] = np.nan

# merge
df = train.append(test)

In [68]:
# datetimeへ変換
df['first_active_month'] = pd.to_datetime(df['first_active_month'])

# datetime features
df['month'] = df['first_active_month'].dt.month.fillna(0).astype(int).astype(object)
df['year'] = df['first_active_month'].dt.year.fillna(0).astype(int).astype(object)
df['month_year'] = df['month'].astype(str)+'_'+df['year'].astype(str)

# features1~3
df['feature_1'] = df['feature_1'].astype(object)
df['feature_2'] = df['feature_2'].astype(object)

# one hot encoding
df, cols = one_hot_encoder(df, nan_as_category=False)

In [40]:
df.groupby('feature_1')['target'].mean()

feature_1
1   -0.315476
2   -0.350452
3   -0.377533
4   -0.404652
5   -0.499879
Name: target, dtype: float64

In [41]:
df.groupby('feature_2')['target'].mean()

feature_2
1   -0.398675
2   -0.332899
3   -0.501889
Name: target, dtype: float64

In [46]:
df.groupby('feature_3')['target'].mean()

feature_3
0   -0.357939
1   -0.421057
Name: target, dtype: float64

In [51]:
df.dtypes

first_active_month    datetime64[ns]
feature_1                      int64
feature_2                      int64
feature_3                      int64
target                       float64
dtype: object

,first_active_month,feature_3,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,...,month_year_8_2014,month_year_8_2015,month_year_8_2016,month_year_8_2017,month_year_9_2012,month_year_9_2013,month_year_9_2014,month_year_9_2015,month_year_9_2016,month_year_9_2017
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06-01,1,-0.820283,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
C_ID_3d0044924f,2017-01-01,0,0.392913,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_d639edf6cd,2016-08-01,0,0.688056,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
C_ID_186d6a6901,2017-09-01,0,0.142495,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
C_ID_cdbd2c0db2,2017-11-01,0,-0.159749,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_0894217f2f,2016-09-01,0,0.871585,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
C_ID_7e63323c00,2016-12-01,1,0.230129,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
C_ID_dfa21fc124,2017-09-01,1,2.135850,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
C_ID_fe0fdac8ea,2017-08-01,0,-0.065406,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [71]:
loadpkl('../output/test_df.pkl')

,first_active_month,feature_3,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,...,month_year_8_2014,month_year_8_2015,month_year_8_2016,month_year_8_2017,month_year_9_2012,month_year_9_2013,month_year_9_2014,month_year_9_2015,month_year_9_2016,month_year_9_2017
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_0ab67a22ab,2017-04-01,1,NaN,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_130fd0cbdd,2017-01-01,0,NaN,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_b709037bc5,2017-08-01,1,NaN,0,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
C_ID_d27d835a9f,2017-12-01,0,NaN,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_2b5e3df5c2,2015-12-01,1,NaN,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_5814b4f13c,2017-07-01,1,NaN,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_a1b3c75277,2017-10-01,0,NaN,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
C_ID_f7cada36d3,2017-02-01,1,NaN,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
C_ID_9d2bc8dfc4,2017-07-01,0,NaN,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
